CNN text classification with one-hot encoding 

Based on: https://cnvrg.io/cnn-sentence-classification/

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn
import pickle

%matplotlib inline

#Import module to split the datasets
from sklearn.model_selection import train_test_split
# Import modules to evaluate the metrics
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,roc_auc_score,roc_curve,auc

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Thesis/Data/Filled Datasets/Second dataset (Kaggle)/Preprocessed/')
train = pd.read_csv("Constraint_Train_fullprep.csv")
test = pd.read_csv("Constraint_Test_fullprep_with_labels.csv",index_col=0)
val = pd.read_csv('Constraint_Val_fullprep.csv')
test = test.drop(["id"], axis=1)
test['label'] = test['label'].map({'real':0,'fake':1})
print('train',train.head(n=3))
print('test',test.head(n=3))
print('val',val.head(n=3))
                  

train                                                tweet  label
0  cdc currently reports deaths general discrepan...      0
1  states reported deaths small rise last tuesday...      0
2  politically correct woman almost uses pandemic...      1
test                                                tweet  label
0  daily update published states reported k tests...      0
1                                 alfalfa cure covid      1
2      president trump asked would catch coronavirus      1
val                                                tweet  label
0  chinese converting islam realising muslim affe...      1
1  people diamond princess cruise ship initially ...      1
2       covid caused bacterium virus treated aspirin      1


In [ ]:
X_train= train['tweet']
y_train= train['label']
X_test = test['tweet']
y_test = test['label']
X_val = val['tweet']
y_val = val['label']

In [ ]:
#we need the next code because the one-hot encoding below otherwise doesn't work
for i in range(len(X_train)): 
    X_train.iloc[i] = str(X_train.iloc[i])

for i in range(len(X_val)): 
    X_val.iloc[i] = str(X_val.iloc[i])

for i in range(len(X_test)): 
    X_test.iloc[i] = str(X_test.iloc[i])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import SMOTE

vocab_size = 5000

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]
X_val = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_val]

In [ ]:
train.loc[33].tweet

'cdc recommends mothers stop breastfeeding boost vaccine efficacy'

In [ ]:
X_train[33]

array([3689, 2835, 2490, 2645,  208, 1779,  508, 1922,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0], dtype=int32)

In [ ]:
max_length = 100
X_train = pad_sequences(X_train, maxlen=max_length, padding='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post')
X_val = pad_sequences(X_val, maxlen=max_length, padding='post')

In [ ]:
smote = SMOTE()
X_train, y_train = smote.fit_resample(X_train,y_train)

In [ ]:
import tensorflow 

In [ ]:
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D, Conv1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding

model = Sequential([
    Embedding(5000, 8, input_length=100),
    Conv1D(128, 5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 8)            40000     
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           5248      
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 46,549
Trainable params: 46,549
Non-trainable params: 0
____________________________________________________

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
checkpoint_path = "/home/lowie/data/rnn_check/"

monitor = 'val_precision'
keras_callbacks   = [
      EarlyStopping(monitor=monitor, patience=30, mode='max', min_delta=0.0001),
      ModelCheckpoint(checkpoint_path, monitor=monitor, save_best_only=True, mode='max')
]
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tensorflow.keras.metrics.Precision()])

In [ ]:
history = model.fit(X_train, y_train, epochs=500, validation_data=(X_val, y_val),callbacks=keras_callbacks)

Epoch 1/500
210/210 [==============================] - 4s 17ms/step - loss: 0.4777 - precision: 0.8428 - val_loss: 0.2786 - val_precision: 0.8692
Epoch 2/500
210/210 [==============================] - 2s 12ms/step - loss: 0.2138 - precision: 0.9093 - val_loss: 0.2503 - val_precision: 0.8946
Epoch 3/500
210/210 [==============================] - 2s 8ms/step - loss: 0.1235 - precision: 0.9535 - val_loss: 0.2633 - val_precision: 0.8851
Epoch 4/500
210/210 [==============================] - 2s 8ms/step - loss: 0.0700 - precision: 0.9773 - val_loss: 0.3026 - val_precision: 0.8849
Epoch 5/500
210/210 [==============================] - 2s 8ms/step - loss: 0.0386 - precision: 0.9879 - val_loss: 0.3418 - val_precision: 0.8903
Epoch 6/500
210/210 [==============================] - 3s 12ms/step - loss: 0.0203 - precision: 0.9938 - val_loss: 0.3922 - val_precision: 0.8981
Epoch 7/500
210/210 [==============================] - 2s 8ms/step - loss: 0.0102 - precision: 0.9970 - val_loss: 0.4273 - val_

In [ ]:
loss, accuracy = model.evaluate(X_test,y_test)
print('Testing Accuracy is {} '.format(accuracy*100))

67/67 [==============================] - 0s 4ms/step - loss: 0.9609 - precision: 0.8657
Testing Accuracy is 86.57004833221436 


In [ ]:
y_pred = model.predict(X_test)
y_pred = y_pred.round()

print(metrics.classification_report(y_test, y_pred,  digits=5))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0    0.88778   0.87589   0.88180      1120
           1    0.86570   0.87843   0.87202      1020

    accuracy                        0.87710      2140
   macro avg    0.87674   0.87716   0.87691      2140
weighted avg    0.87726   0.87710   0.87714      2140

[[981 139]
 [124 896]]


**SAVE THE CNN STRUCTURE**

In [ ]:
model.save('netron_CNN.h5')

In [ ]:
from keras.models import model_from_json
os.chdir('/content/drive/MyDrive/Thesis/Models/Replicated models /Simple models /CNN/')
# serialize model to JSON
model_json = model.to_json()
with open("CNN.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
from keras.models import model_from_json

# load json and create model
os.chdir('/content/drive/MyDrive/Thesis/Models/Replicated models /Simple models /CNN/')
json_file = open('CNN.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("netron_CNN.h5")
print("Loaded CNN from disk")

Loaded CNN from disk
